# SysML v2 Test Suite

This can notebook can be cleared using menu `Edit --> Clear All Outputs` and then all cells can be
run using menu `Run --> Run All Cells`.

**WARNING**: This file is generated, all modifications here will be ignored.


## Base Example Directory: ../../sysml/src/examples/Analysis Examples

## File: AnalysisAnnotation.sysml

In [1]:
package AnalysisAnnotation {
	import ScalarValues::Real;
	import AnalysisTooling::*;
	import ISQ::*;
	
	action def ComputeDynamics {
		metadata ToolExecution {
			toolName = "ModelCenter";
			uri = "aserv://localhost/Vehicle/Equation1";
		}
			
		in dt : TimeValue             { @ToolVariable { name = "deltaT"; } }
		in whlpwr : PowerValue        { @ToolVariable { name = "power"; } }
		in Cd : Real                  { @ToolVariable { name = "C_D"; } }
		in Cf: Real                   { @ToolVariable { name = "C_F"; } }
		in tm : MassValue             { @ToolVariable { name = "mass"; } }
		in v_in : SpeedValue       { @ToolVariable { name = "v0"; } }
		in x_in : LengthValue         { @ToolVariable { name = "x0"; } }
		
		out a_out : AccelerationValue { @ToolVariable { name = "a"; } }
		out v_out : SpeedValue     { @ToolVariable { name = "v"; } }
		out x_out : LengthValue       { @ToolVariable { name = "x"; } }
			
	}

}

Package AnalysisAnnotation (02a32c10-029d-4210-b660-aeef7b10e76c)


## File: Dynamics.sysml

In [2]:
package Dynamics {
	import ScalarValues::Real;
	import ISQ::*;
	
	// Function definitions
	
	calc def Power
		(in whlpwr : PowerValue, in Cd : Real, in Cf : Real, in tm : MassValue, in v : SpeedValue )
		return tp : PowerValue {
			
		whlpwr - Cd * v - Cf * tm * v
	}
	
	calc def Acceleration (dt : TimeValue, tm : MassValue, tp: PowerValue) a : AccelerationValue { 
		tp * dt * tp
	}
	
	calc def Velocity (dt : TimeValue, v0 : SpeedValue, a : AccelerationValue) v : SpeedValue = v0 + a * dt;
 	
	calc def Position (dt : TimeValue, x0 : LengthValue, v : SpeedValue) : LengthValue = x0 + v * dt;
	

	// Analysis action def
	
	action def StraightLineVehicleDynamics {
		
		in attribute dt : TimeValue;
		in attribute whlpwr : PowerValue;
		in attribute Cd : Real;
		in attribute Cf: Real;
		in attribute tm : MassValue;
		in attribute v_in : SpeedValue;
		in attribute x_in : LengthValue;
		
		out attribute a_out : AccelerationValue;
		out attribute v_out : SpeedValue;
		out attribute x_out : LengthValue;
			
		assert constraint {
			attribute tp : PowerValue;
			
			tp == Power(whlpwr, Cd, Cf, tm, v_in) &
			a_out == Acceleration(dt, tm, tp) &
			v_out == Velocity(dt, v_in, a_out) &
			x_out == Position(dt, x_in, v_in)
		}
	}
	

	// Analysis actions
	
	action dyn1 : StraightLineVehicleDynamics {
		attribute tp : PowerValue = Power(whlpwr, Cd, Cf, tm, v_in);
		
		out attribute :>> a_out : AccelerationValue = Acceleration(dt, tm, tp);
		out attribute :>> v_out : SpeedValue = Velocity(dt, v_in, a_out);
		out attribute :>> x_out : LengthValue = Position(dt, x_in, v_in);
	}	
	
	action dyn2 : StraightLineVehicleDynamics {
		calc acc : Acceleration (
			dt = dyn2::dt, 
			tm = dyn2::tm, 
			tp = Power(whlpwr, Cd, Cf, tm, v_in)
		) return a = a_out;
		
		calc vel : Velocity (
			dt = dyn2::dt, 
			v0 = dyn2::v_in, 
			a = acc::a
		) return v = v_out;
		
		calc pos : Position (
			dt = dyn2::dt, 
			x0 = dyn2::x_in, 
			v0 = vel::v
		) return x = x_out;		
	}
}

Package Dynamics (c0fba5e0-acdc-4261-8626-daa5d0820efd)


## File: Turbojet Stage Analysis.sysml

In [3]:
package 'Turbojet Stage Analysis' {
	import Quantities::ScalarQuantityValue;
	import UnitsAndScales::DimensionOneValue;
	import ISQ::*;
	
	package 'Thermodynamic Functions' {
	    calc def 'Ideal Gas Law' (in rho, in R_bar, in T) return p {
	        rho * R_bar * T
	    }
	    
	    // The "in" and "return" direction markers are optional.
	    calc def 'Reversible Adiabatic Compression Density' (rho_1, p_1, p_2, gamma) rho_2 {
	        rho_1 * (p_2 / p_1)**(1/gamma)
	    }
	    
	    // Shorthand for "mathematical" function definition (with no steps but the returned expression)
	    calc def 'Reversible Adiabatic Compression Temperature' (T_1, p_1, p_2, gamma) 
	    	T_2 = T_1 * (p_2 / p_1)**((gamma - 1) / gamma);
	    
	    calc def 'Total Pressure' (P_static, rho, V) = 1/2 * rho * V ** 2 + P_static;
	    
	    // Showing explicit parameter typing
	    calc def 'Total Temperature' (T_static : TemperatureValue, Cp : DimensionOneValue, V : VolumeValue) : TemperatureValue 
	    	= 1/(2 * Cp) * V ** 2 + T_static;
	    
	    calc def 'Total Enthalpy' (h_total, h_static, V) H_total {
	    	1/2 * V ** 2 + h_static
	    }
	}
	
	package 'Thermodynamics Structure' {
	    part def 'Ideal Gas Parcel' {
	        comment
	            /*
	            The parcel is an infinitesimal volume used to analyze points in a flow
	            */
	        attribute 'Molar Mass';
	        attribute 'Density';
	        attribute 'Pressure';
	        attribute 'Temperature';
	        attribute 'Enthalpy';
	        attribute 'Specific Gas Constant';
	    }
	    
	    part def 'Moving Ideal Gas Parcel' specializes 'Ideal Gas Parcel' {
	        comment about 'Stagnation Pressure'
	            /*
	            Stagnation pressure is the pressure of the parcel if the kinetic energy defined by its
	            velocity in a given coordinate frame is converted to gas internal energy through deceleration
	            to a velocity that matches the current frame.
	            */
	        attribute 'Stagnation Pressure';
	        attribute 'Stagnation Temperature';
	        attribute 'Stagnation Enthalpy';
	        
	        comment about 'Static Pressure'
	            /*
	            Static pressure is the pressure of the parcel as it moves
	            */
	        attribute 'Static Pressure' redefines 'Ideal Gas Parcel'::'Pressure';
	        attribute 'Static Temperature' redefines 'Ideal Gas Parcel'::'Temperature';
	        attribute 'Static Enthalpy' redefines 'Ideal Gas Parcel'::'Enthalpy';
	    }
	    
	    action def 'Thermodynamic Process'; // need start and end shots to show beginning and end attributes
	    
	    action def 'Adiabatic Process' specializes 'Thermodynamic Process' {
	        /*
	        Thermodynamic process typically have their states defined at beginning and end
	        of the process (since these starts are path-independent)
	        */
	        action 'Stage 1' :>> start;
	        action 'Stage 2' :>> done;
	    }
	    
	    action def 'Reversible Adiabatic Process' specializes 'Adiabatic Process';
	}
	
	package 'Low-Pressure Compressor Analysis' {
	    
	    part 'Analysis Context' {
	        import 'Thermodynamic Functions'::*;
	        
	        part 'Inlet Gas' : 'Thermodynamics Structure'::'Moving Ideal Gas Parcel' {
	        	// Explicit binding notation
	        	calc 'Solve for Pressure1' : 'Ideal Gas Law' (rho, R_bar, T) p;
	        	bind 'Density' = 'Solve for Pressure1'::rho;
	        	bind 'Specific Gas Constant' = 'Solve for Pressure1'::R_bar;
	        	bind 'Static Temperature' = 'Solve for Pressure1'::T;
	        	bind 'Static Pressure' = 'Solve for Pressure1'::p;	        	
	        	
	        	// Shorthand parameter binding notation (also using optional "in" and "return")
	            calc 'Solve for Pressure2' : 'Ideal Gas Law'(
	                in rho = 'Density',
	                in R_bar = 'Specific Gas Constant',
	                in T = 'Static Temperature'
	            ) return p = 'Static Pressure';
	            
	            // Invocation expression notation
	            attribute :>> 'Static Pressure' = 'Ideal Gas Law'('Density', 'Specific Gas Constant', 'Static Temperature');

	            // Equation as binding (possible future notation) 
	            // eqn 'Static Pressure' = 'Ideal Gas Law'('Density', 'Specific Gas Constant', 'Static Temperature');
	            
	            // Equation as a constraint (note "==")
	            constraint { 'Static Pressure' == 'Ideal Gas Law'('Density', 'Specific Gas Constant', 'Static Temperature') }
	        }
	    }
	}	
	
}

Package Turbojet Stage Analysis (40643fad-88ad-4553-8620-2cd916802f5e)


## File: Vehicle Analysis Demo.sysml

In [4]:
package 'Vehicle Analysis Demo' {
	package VehicleQuantities {
	    import ScalarValues::*;
	    import Quantities::*;
	    import UnitsAndScales::*;
	    import ISQ::*;
	    import USCustomaryUnits::*;
	    
	    attribute def DistancePerVolumeUnit :> DerivedUnit {
	    	private attribute distancePF: QuantityPowerFactor[1] { :>> quantity = isq::L; :>> exponent = 1; }
	        private attribute volumePF: QuantityPowerFactor[1] { :>> quantity = isq::L; :>> exponent = -3; }
	        attribute :>> quantityDimension { :>> quantityPowerFactors = {distancePF, volumePF}; }
	    }

	    attribute def DistancePerVolumeValue :> QuantityValue {
	        :>> num : Real;
	        :>> mRef : DistancePerVolumeUnit;
	    }
	    
	    attribute gallon : VolumeUnit = 231.0 * 'in' ** 3;
	    attribute mpg : DistancePerVolumeUnit = 'mi' / gallon;
	}
	
	package VehicleModel {
	    import VehicleQuantities::*;
	    
	    item def Fuel;
	        
	    port def FuelPort {
	        out item fuel: Fuel;
	    }
	    
	    part def FuelTank {
	        attribute volumeMax : VolumeValue;
	        attribute fuelVolume : VolumeValue;
	        attribute fuelLevel : Real = fuelVolume / volumeMax;
	        
	        port fuelInPort : ~FuelPort;
	        port fuelOutPort : FuelPort;
	    }
	    
	    part def Wheel {
	        attribute diameter : LengthValue;
	    }
	
	    part def Vehicle {
	        attribute mass : MassValue;
	        attribute cargoMass : MassValue;
	        
	        attribute wheelDiameter : LengthValue;
	        attribute driveTrainEfficiency : Real;
	        
	        attribute fuelEconomy_city : DistancePerVolumeValue;
	        attribute fuelEconomy_highway : DistancePerVolumeValue;
	
	        port fuelInPort : ~FuelPort;
	    }
	    
	    part vehicle_c1 : Vehicle {
	        port :>> fuelInPort {
	            in item :>> fuel;
	        }
	
	        part fuelTank : FuelTank {
	            port :>> fuelInPort {
	                in item :>> fuel;
	            }
	        }
	
	        bind fuelInPort::fuel = fuelTank::fuelInPort::fuel;
	
	        part wheel : Wheel[4] {
	            :>> diameter = wheelDiameter;
	        }
	    }
	}
	
	package FuelEconomyRequirementsModel {
	    import VehicleQuantities::*;
	
	    requirement def FuelEconomyRequirement {
	        attribute actualFuelEconomy : DistancePerVolumeValue;
	        attribute requiredFuelEconomy : DistancePerVolumeValue;
	
	        require constraint { actualFuelEconomy >= requiredFuelEconomy }
	    }
	
	    requirement cityFuelEconomyRequirement : FuelEconomyRequirement {
	        :>> requiredFuelEconomy = 25@[mpg];
	    }
	
	    requirement highwayFuelEconomyRequirement : FuelEconomyRequirement {
	        :>> requiredFuelEconomy = 30@[mpg];
	    }
	}
	
	package DynamicsModel {
	    import VehicleQuantities::*;
	    
	    calc def Acceleration(p : PowerValue, m : MassValue, v : SpeedValue) : AccelerationValue =
	        p / (m * v);
	    
	    calc def Velocity(v0 : SpeedValue, a : AccelerationValue, dt : TimeValue) : SpeedValue =
	        v0 + a * dt;
	    
	    calc def Position(x0 : LengthValue, v : SpeedValue, dt : TimeValue) : LengthValue =
	        x0 + v * dt;
	    
	    constraint def StraightLineDynamicsEquations(
	        p : PowerValue,
	        m : MassValue,
	        dt : TimeValue,
	        x_i : LengthValue,
	        v_i : SpeedValue,
	        x_f : LengthValue,
	        v_f : SpeedValue,
	        a : AccelerationValue
	    ) {
	        attribute v_avg : SpeedValue = (v_i + v_f)/2;
	
	        a == Acceleration(p, m, v_avg) &
	        v_f == Velocity(v_i, a, dt) &
	        x_f == Position(x_i, v_avg, dt)
	    }
	    
	    action def StraightLineDynamics (
	        in power : PowerValue,
	        in mass : MassValue,
	        in delta_t : TimeValue,
	        in x_in : LengthValue,
	        in v_in : SpeedValue,
	        out x_out : LengthValue,
	        out v_out : SpeedValue,
	        out a_out : AccelerationValue
	    ) {
	        assert constraint dynamics : StraightLineDynamicsEquations (
	            p = power,
	            m = mass,
	            dt = delta_t,
	            x_i = x_in,
	            v_i = v_in,
	            x_f = x_out,
	            v_f = v_out,
	            a = a_out
	        );
	    }
	}
	
	package FuelEconomyAnalysisModel {
	    import VehicleModel::*;
	    import FuelEconomyRequirementsModel::*;
	    import DynamicsModel::*;
	    import BaseFunctions::size;
	    import Collections::SampledFunction;
	    import Collections::SamplePair;
	
		attribute def ScenarioState {
			position : LengthValue;
			velocity : SpeedValue;
		}
		
		attribute def NominalScenario :> SampledFunction {
			attribute def TimeStateRecord :> SamplePair {
				t : TimeValue :>> domainValue;
				s : ScenarioState :>> rangeValue;
			}
			:>> samples : TimeStateRecord;
			n : Natural = size(samples);
		}
		
		analysis def FuelEconomyAnalysis () calculatedFuelEconomy : DistancePerVolumeValue {
			subject vehicle: Vehicle;
			in attribute scenario : NominalScenario;
			in requirement fuelEconomyRequirement : FuelEconomyRequirement;
			
			objective fuelEconomyAnalysisObjective {
				doc 
				/*
				 * The objective of this analysis is to determine whether the
				 * current vehicle design configuration can satisfy the fuel
				 * economy requirement.
				 */
				
				assume constraint {
					vehicle::wheelDiameter == 33@['in'] &
					vehicle::driveTrainEfficiency == 0.4
				}
				
				require fuelEconomyRequirement {
					:>> actualFuelEconomy = calculatedFuelEconomy;
				}
			}
			
		    
			//import scenario::*;
		
			action dynamicsAnalysis(
				in sc: NominalScenario,
				out power : PowerValue[*],
				out acceleration : AccelerationValue[*]	) {
				/*
				 * Solve for the required engine power as a function of time
				 * to support the scenarios.
				 */
				assert constraint straightLineDynamics {
					{1..sc.n-1}->forAll i (
						StraightLineDynamicsEquations (
							p => power[i],
							m => vehicle::mass,
							dt => sc.samples.t[i+1] - sc.samples.t[i],
							x_i => sc.samples.s.position[i],
							v_i => sc.samples.s.velocity[i],
							x_f => sc.samples.s.position[i+1],
							v_f => sc.samples.s.velocity[i+1],
							a => acceleration[i]                    
						)
					)
				}
			}

		    action fuelConsumptionAnalysis(
		        in power : PowerValue[*] = dynamicsAnalysis::power,
		        in acceleration : AccelerationValue[*] = dynamicsAnalysis::acceleration,
		        out fuelEconomy : DistancePerVolumeValue = calculatedFuelEconomy ) {
		        /*
		         * Solve the engine equations to determine how much fuel is
		         * consumed. The engine RPM is a function of the speed of the
		         * vehicle and the gear state.
		         */
	        }
	    }
	}

	
	part vehicleFuelEconomyAnalysisContext {
	    import FuelEconomyAnalysisModel::*;
	
	    requirement vehicleFuelEconomyRequirementsGroup {
	        import FuelEconomyRequirementsModel::*;
	        
	        subject vehicle : Vehicle;
	
	        requirement vehicleFuelEconomyRequirement_city :> cityFuelEconomyRequirement {
	            doc /* The vehicle shall provide a fuel economy that is greater than or equal to
	             * 25 miles per gallon for the nominal city driving scenarios.
	             */
	
	            :>> actualFuelEconomy = vehicle::fuelEconomy_city;
	
	            assume constraint { vehicle::cargoMass == 1000@[lb] }
	        }
	
	        requirement vehicleFuelEconomyRequirement_highway :> highwayFuelEconomyRequirement {
	            doc /* The vehicle shall provide a fuel economy that is greater than or equal to
	             * 30 miles per gallon for the nominal highway driving scenarios.
	             */
	
	            :>> actualFuelEconomy = vehicle::fuelEconomy_highway;
	
	            assume constraint { vehicle::cargoMass == 1000@[lb] }
	        }
	
	    }
	    
	    attribute cityScenario : NominalScenario;
	    attribute highwayScenario : NominalScenario;
	
	    analysis cityFuelEconomyAnalysis : FuelEconomyAnalysis () {
	        subject vehicle = vehicle_c1;
	        attribute :>> scenario = cityScenario;
	        requirement :>> fuelEconomyRequirement = cityFuelEconomyRequirement;
	    }
	
	    analysis highwayFuelEconomyAnalysis : FuelEconomyAnalysis () {
	        subject vehicle = vehicle_c1;
	        attribute :>> scenario = highwayScenario;
	        requirement :>> fuelEconomyRequirement = highwayFuelEconomyRequirement;
	    }
	
	    part vehicle_c1_analysized :> vehicle_c1 {
	        attribute :>> fuelEconomy_city = cityFuelEconomyAnalysis::calculatedFuelEconomy;
	        attribute :>> fuelEconomy_highway = highwayFuelEconomyAnalysis::calculatedFuelEconomy;
	    }
	
	    satisfy vehicleFuelEconomyRequirementsGroup by vehicle_c1_analysized;
	}
}

Package Vehicle Analysis Demo (b5a3c4d5-6919-4733-a8da-02c338a9450b)
